In [1]:
magma_dir = '/home/ubuntu/magma/'
bucket_dir = '/home/ubuntu/s3/'
transformers_dir = '/home/ubuntu/transformers/'
cache_dir = bucket_dir+'.cache/'

## **Fine-tuning**

In [2]:
finetune_script = '"'+transformers_dir+'examples/seq2seq/finetune_trainer.py"'
eval_script = '"'+transformers_dir+'examples/seq2seq/run_eval.py"'

### **Config**

In [3]:
import sys
sys.path.insert(0, magma_dir)
import config

import torch
torch.manual_seed = config.SEED

import wandb
wandb.login()

project_name = 'hps_bart_para_wordembed'
%env WANDB_PROJECT=$project_name

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: marcoabrate (use `wandb login --relogin` to force relogin)


env: WANDB_PROJECT=hps_bart_para_wordembed


### HP search

In [4]:
model_name_or_path = 'sshleifer/distilbart-cnn-12-6'

In [5]:
data_dir = '"'+bucket_dir+'datasets/karger_books_para_wordembed/bart/st/"'

output_dir = '"'+bucket_dir+'fine-tuning/hps_bart_para_wordembed"'

log_dir = output_dir + '/logs'

In [6]:
hp_search_checkpoint_dir = output_dir + '/ray_checkpoints'

hp_search_log_dir = output_dir + '/ray_results'

In [7]:
from transformers import AutoConfig
model_config = AutoConfig.from_pretrained(model_name_or_path, use_cache=False)
model_config.min_length = config.ONE_BULLET_MIN_LEN
model_config.max_length = config.ONE_BULLET_MAX_LEN
model_config.num_beams = 2

model_config.task_specific_params['summarization']['min_length'] = config.ONE_BULLET_MIN_LEN
model_config.task_specific_params['summarization']['max_length'] = config.ONE_BULLET_MAX_LEN
model_config.task_specific_params['summarization']['num_beams'] = 2
model_config_dir = '"'+bucket_dir+'fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_config"'
model_config.save_pretrained(model_config_dir[1:-1])

In [8]:
!python3 $finetune_script \
--model_name_or_path $model_name_or_path \
--config_name $model_config_dir \
--tokenizer_name $model_name_or_path \
--cache_dir $cache_dir \
--data_dir $data_dir \
--freeze_embeds \
--hp_search \
--hp_search_checkpoint_dir $hp_search_checkpoint_dir --hp_search_db_host '0.0.0.0' \
--hp_search_name $project_name --hp_search_log_dir $hp_search_log_dir \
--warmup_steps 0 \
--fp16 \
--label_smoothing_factor 0.1 \
--sortish_sampler \
--task summarization \
--max_target_length $config.ONE_BULLET_MAX_LEN \
--val_max_target_length $config.ONE_BULLET_MAX_LEN \
--max_steps 750 \
--logging_steps 20 --logging_first_step \
--per_device_train_batch_size 2 --per_device_eval_batch_size 8 \
--evaluation_strategy steps --eval_steps 50 --eval_beams 2 \
--predict_with_generate \
--save_steps 5000 \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED \
--run_name $output_dir

02/10/2021 17:07:10 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: True
02/10/2021 17:07:10 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='/home/ubuntu/s3/fine-tuning/hps_bart_para_wordembed', overwrite_output_dir=True, do_train=False, do_eval=None, do_predict=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=750, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_steps=0, logging_dir='runs/Feb10_17-07-10_ip-172-31-39-35', logging_first_step=True, logging_steps=20, save_steps=5000, save_total_limit=None, n

[INFO|modeling_utils.py:1143] 2021-02-10 17:07:39,484 >> All model checkpoint weights were used when initializing BartForConditionalGeneration.

[INFO|modeling_utils.py:1152] 2021-02-10 17:07:39,484 >> All the weights of BartForConditionalGeneration were initialized from the model checkpoint at sshleifer/distilbart-cnn-12-6.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BartForConditionalGeneration for predictions without further training.
02/10/2021 17:07:39 - INFO - utils -   setting model.config to task specific params for summarization:
 {'early_stopping': True, 'length_penalty': 2.0, 'max_length': 150, 'min_length': 10, 'no_repeat_ngram_size': 3, 'num_beams': 2}
02/10/2021 17:07:39 - INFO - utils -   note: command line args may override some of these
[INFO|modeling_utils.py:1027] 2021-02-10 17:07:40,534 >> loading weights file https://huggingface.co/sshleifer/distilbart-cnn-12-6/resolve/main/pytorch_model.bin from cache at /hom

== Status ==
Memory usage on this node: 11.3/15.3 GiB
Using FIFO scheduling algorithm.
Resources requested: 3/4 CPUs, 1/1 GPUs, 0.0/6.01 GiB heap, 0.0/2.05 GiB objects (0/1.0 accelerator_type:T4)
Result logdir: /home/ubuntu/s3/fine-tuning/hps_bart_para_wordembed/ray_results/hps_bart_para_wordembed
Number of trials: 2/12 (1 PENDING, 1 RUNNING)
+--------------------+----------+--------------------+-------------------------------+-----------------+-------------+
| Trial name         | status   | loc                |   gradient_accumulation_steps |   learning_rate |   objective |
|--------------------+----------+--------------------+-------------------------------+-----------------+-------------|
| _inner_8ee26_00000 | RUNNING  | 172.31.39.35:10487 |                             8 |           1e-05 |     613.647 |
| _inner_8ee26_00001 | PENDING  |                    |                            32 |           1e-05 |             |
+--------------------+----------+--------------------+------

== Status ==
Memory usage on this node: 11.3/15.3 GiB
Using FIFO scheduling algorithm.
Resources requested: 3/4 CPUs, 1/1 GPUs, 0.0/6.01 GiB heap, 0.0/2.05 GiB objects (0/1.0 accelerator_type:T4)
Result logdir: /home/ubuntu/s3/fine-tuning/hps_bart_para_wordembed/ray_results/hps_bart_para_wordembed
Number of trials: 2/12 (1 PENDING, 1 RUNNING)
+--------------------+----------+--------------------+-------------------------------+-----------------+-------------+
| Trial name         | status   | loc                |   gradient_accumulation_steps |   learning_rate |   objective |
|--------------------+----------+--------------------+-------------------------------+-----------------+-------------|
| _inner_8ee26_00000 | RUNNING  | 172.31.39.35:10487 |                             8 |           1e-05 |     619.284 |
| _inner_8ee26_00001 | PENDING  |                    |                            32 |           1e-05 |             |
+--------------------+----------+--------------------+------

== Status ==
Memory usage on this node: 11.3/15.3 GiB
Using FIFO scheduling algorithm.
Resources requested: 3/4 CPUs, 1/1 GPUs, 0.0/6.01 GiB heap, 0.0/2.05 GiB objects (0/1.0 accelerator_type:T4)
Result logdir: /home/ubuntu/s3/fine-tuning/hps_bart_para_wordembed/ray_results/hps_bart_para_wordembed
Number of trials: 2/12 (1 PENDING, 1 RUNNING)
+--------------------+----------+--------------------+-------------------------------+-----------------+-------------+
| Trial name         | status   | loc                |   gradient_accumulation_steps |   learning_rate |   objective |
|--------------------+----------+--------------------+-------------------------------+-----------------+-------------|
| _inner_8ee26_00000 | RUNNING  | 172.31.39.35:10487 |                             8 |           1e-05 |     666.481 |
| _inner_8ee26_00001 | PENDING  |                    |                            32 |           1e-05 |             |
+--------------------+----------+--------------------+------

...

Result for _inner_8ee26_00010:
  date: 2021-02-11_08-38-03
  done: false
  epoch: 9.66
  eval_gen_len: 28.0
  eval_loss: 6.51438045501709
  eval_rouge1_fmeasure: 10.16
  eval_rouge1_precision: 12.389999999999999
  eval_rouge1_recall: 9.41
  eval_rouge2_fmeasure: 0.22999999999999998
  eval_rouge2_precision: 0.29
  eval_rouge2_recall: 0.21
  eval_rougeL_fmeasure: 8.450000000000001
  eval_rougeL_precision: 10.09
  eval_rougeL_recall: 7.9399999999999995
  eval_rougeLsum_fmeasure: 8.61
  eval_rougeLsum_precision: 10.36
  eval_rougeLsum_recall: 8.06
  eval_runtime: 83.4226
  eval_samples_per_second: 3.189
  eval_sentence_distilroberta_cosine: 26.84316337108612
  eval_w2v_cosine: 83.40513110160828
  experiment_id: 22638209cfa64fb589583036aa5e47eb
  hostname: ip-172-31-39-35
  iterations_since_restore: 6
  node_ip: 172.31.39.35
  objective: 283.0598944726944
  pid: 19815
  time_since_restore: 1606.8855247497559
  time_this_iter_s: 249.85753226280212
  time_total_s: 1606.8855247497559
  timesta

== Status ==
Memory usage on this node: 11.3/15.3 GiB
Using FIFO scheduling algorithm.
Resources requested: 3/4 CPUs, 1/1 GPUs, 0.0/6.01 GiB heap, 0.0/2.05 GiB objects (0/1.0 accelerator_type:T4)
Result logdir: /home/ubuntu/s3/fine-tuning/hps_bart_para_wordembed/ray_results/hps_bart_para_wordembed
Number of trials: 12/12 (1 PENDING, 1 RUNNING, 10 TERMINATED)
+--------------------+------------+--------------------+-------------------------------+-----------------+-------------+
| Trial name         | status     | loc                |   gradient_accumulation_steps |   learning_rate |   objective |
|--------------------+------------+--------------------+-------------------------------+-----------------+-------------|
| _inner_8ee26_00010 | RUNNING    | 172.31.39.35:19815 |                            32 |          0.0005 |     337.212 |
| _inner_8ee26_00011 | PENDING    |                    |                            64 |          0.0005 |             |
| _inner_8ee26_00000 | TERMINATED 

Result for _inner_8ee26_00010:
  date: 2021-02-11_09-00-31
  done: false
  epoch: 17.72
  eval_gen_len: 37.0
  eval_loss: 7.031271934509277
  eval_rouge1_fmeasure: 13.76
  eval_rouge1_precision: 14.63
  eval_rouge1_recall: 14.24
  eval_rouge2_fmeasure: 0.36
  eval_rouge2_precision: 0.38
  eval_rouge2_recall: 0.37
  eval_rougeL_fmeasure: 10.05
  eval_rougeL_precision: 10.58
  eval_rougeL_recall: 10.51
  eval_rougeLsum_fmeasure: 10.23
  eval_rougeLsum_precision: 10.83
  eval_rougeLsum_recall: 10.65
  eval_runtime: 100.851
  eval_samples_per_second: 2.638
  eval_sentence_distilroberta_cosine: 30.47771155834198
  eval_w2v_cosine: 70.6612765789032
  experiment_id: 22638209cfa64fb589583036aa5e47eb
  hostname: ip-172-31-39-35
  iterations_since_restore: 11
  node_ip: 172.31.39.35
  objective: 311.2179881372452
  pid: 19815
  time_since_restore: 2954.6277208328247
  time_this_iter_s: 266.6467139720917
  time_total_s: 2954.6277208328247
  timestamp: 1613034031
  timesteps_since_restore: 0
  tra

Result for _inner_8ee26_00010:
  date: 2021-02-11_09-13-40
  done: false
  epoch: 22.56
  eval_gen_len: 21.0
  eval_loss: 7.192877292633057
  eval_rouge1_fmeasure: 11.93
  eval_rouge1_precision: 15.39
  eval_rouge1_recall: 10.51
  eval_rouge2_fmeasure: 0.45999999999999996
  eval_rouge2_precision: 0.63
  eval_rouge2_recall: 0.4
  eval_rougeL_fmeasure: 9.73
  eval_rougeL_precision: 12.43
  eval_rougeL_recall: 8.68
  eval_rougeLsum_fmeasure: 9.86
  eval_rougeLsum_precision: 12.620000000000001
  eval_rougeLsum_recall: 8.77
  eval_runtime: 59.2279
  eval_samples_per_second: 4.491
  eval_sentence_distilroberta_cosine: 20.405031740665436
  eval_w2v_cosine: 94.28302645683289
  experiment_id: 22638209cfa64fb589583036aa5e47eb
  hostname: ip-172-31-39-35
  iterations_since_restore: 14
  node_ip: 172.31.39.35
  objective: 279.8169581974983
  pid: 19815
  time_since_restore: 3744.220279932022
  time_this_iter_s: 228.43896961212158
  time_total_s: 3744.220279932022
  timestamp: 1613034820
  timestep

Result for _inner_8ee26_00011:
  date: 2021-02-11_09-27-21
  done: false
  epoch: 3.31
  eval_gen_len: 44.0
  eval_loss: 4.164558410644531
  eval_rouge1_fmeasure: 14.57
  eval_rouge1_precision: 13.33
  eval_rouge1_recall: 17.84
  eval_rouge2_fmeasure: 1.25
  eval_rouge2_precision: 1.09
  eval_rouge2_recall: 1.6099999999999999
  eval_rougeL_fmeasure: 10.549999999999999
  eval_rougeL_precision: 9.629999999999999
  eval_rougeL_recall: 12.98
  eval_rougeLsum_fmeasure: 10.77
  eval_rougeLsum_precision: 9.9
  eval_rougeLsum_recall: 13.19
  eval_runtime: 123.6113
  eval_samples_per_second: 2.152
  eval_sentence_distilroberta_cosine: 21.824389696121216
  eval_w2v_cosine: 57.94774889945984
  experiment_id: a5b95ebdf11745b1aa3ea151bcf16458
  hostname: ip-172-31-39-35
  iterations_since_restore: 1
  node_ip: 172.31.39.35
  objective: 322.24543859558105
  pid: 20485
  time_since_restore: 570.4673585891724
  time_this_iter_s: 570.4673585891724
  time_total_s: 570.4673585891724
  timestamp: 16130356